# Imports

In [1]:
from datetime import datetime
import numpy as np
import os
import pandas as pd
from pandas import json_normalize
from sklearn.model_selection import train_test_split
import time
from time import gmtime, strftime, sleep
from io import StringIO
import requests

import sagemaker
from sagemaker.feature_store.feature_group import FeatureGroup
import boto3

c:\Users\sadai\Desktop\AQI-Prediction\.venv\lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[07/01/25 11:22:12] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=11912;file://c:\Users\sadai\Desktop\AQI-Prediction\.venv\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=862017;file://c:\Users\sadai\Desktop\AQI-Prediction\.venv\lib\site-packages\botocore\credentials.py#1352\1352]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\sadai\AppData\Local\sagemaker\sagemaker\config.yaml


In [2]:
os.chdir('..')
os.getcwd()

'c:\\Users\\sadai\\Desktop\\AQI-Prediction'

In [4]:
AQOPEN_API_KEY = '4f3f6e2b1360980aec87f727b4e6d7fb70a6169f'

cities = ['los_angeles', 'boston', 'san_francisco', 'shanghai',
           'seoul', 'paris', 'london']

# Merge Historical Data

In [ ]:
la20 = pd.read_csv('data/raw_data/weather/la_20.csv')
la21 = pd.read_csv('data/raw_data/weather/la_21.csv')
la22 = pd.read_csv('data/raw_data/weather/la_22.csv')
la23 = pd.read_csv('data/raw_data/weather/la_23.csv')
la24 = pd.read_csv('data/raw_data/weather/la_24.csv')

la_air_quality = pd.read_csv('data/raw_data/air_quality/los_angeles.csv')
la_air_quality.columns = la_air_quality.columns.str.strip()
la_air_quality['date'] = pd.to_datetime(la_air_quality['date'])#, format='%Y-%m-%d')

merged_weather = pd.concat([la20, la21, la22, la23, la24], ignore_index=True)
merged_weather.rename(columns={'datetime': 'date'}, inplace=True)
merged_weather['date'] = pd.to_datetime(merged_weather['date'], format='%Y-%m-%d')
merged_df = pd.merge(merged_weather, la_air_quality, on='date', how='inner')

In [3]:
#adding in missing 2025 data
la251 = pd.read_csv('data/raw_data/weather/la_25_1.csv')
la252 = pd.read_csv('data/raw_data/weather/la_25_2.csv')

la_air_quality = pd.read_csv('data/raw_data/air_quality/los-angeles-north main street-air-quality.csv')
la_air_quality['date'] = pd.to_datetime(la_air_quality['date'])#, format='%Y-%m-%d')

merged_weather = pd.concat([la251, la252], ignore_index=True)
merged_weather.rename(columns={'datetime': 'date'}, inplace=True)
merged_weather['date'] = pd.to_datetime(merged_weather['date'], format='%Y-%m-%d')
merged25_df = pd.merge(merged_weather, la_air_quality, on='date', how='inner')

In [5]:
# merged_df.head()
merged25_df.head()

,name,date,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,conditions,description,icon,stations,pm25,pm10,o3,no2,so2,co
0,los angeles,2025-01-01,61.9,46.2,54.4,61.9,44.2,54.3,46.5,75.7,...,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KHHR,72295023174,KBUR,SE428,72295603167,F1624,...",,34,23,21,,8
1,los angeles,2025-01-02,72.3,47.1,56.4,72.3,47.1,56.4,42.4,62.5,...,Clear,Clear conditions throughout the day.,clear-day,"KHHR,72295023174,KBUR,SE428,72295603167,F1624,...",95,33,15,34,,8
2,los angeles,2025-01-03,61.9,49.5,54.3,61.9,49.3,54.3,47.7,79.0,...,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KHHR,72295023174,KBUR,SE428,72295603167,F1624,...",67,37,21,30,,10
3,los angeles,2025-01-04,61.9,48.8,55.5,61.9,48.8,55.5,46.4,72.2,...,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"KHHR,72295023174,KBUR,SE428,72295603167,F1624,...",80,34,25,18,,5
4,los angeles,2025-01-05,74.1,46.9,58.8,74.1,46.9,58.8,36.2,47.2,...,Clear,Clear conditions throughout the day.,clear-day,"KHHR,72295023174,KBUR,SE428,72295603167,F1624,...",88,21,29,23,,6


# Data Preparation

In [6]:
# merged_df.drop(columns=['name', 'description', 'icon', 'stations', 'sunrise', 'sunset', 'severerisk', 'preciptype', 'pm10', 'o3', 'no2', 'so2', 'co'], inplace=True)
merged25_df.drop(columns=['name', 'description', 'icon', 'stations', 'sunrise', 'sunset', 'severerisk', 'preciptype', 'pm10', 'o3', 'no2', 'so2', 'co'], inplace=True)

In [9]:
merged25_df.dtypes

date                datetime64[ns]
tempmax                    float64
tempmin                    float64
temp                       float64
feelslikemax               float64
feelslikemin               float64
feelslike                  float64
dew                        float64
humidity                   float64
precip                     float64
precipprob                   int64
precipcover                float64
snow                         int64
snowdepth                    int64
windgust                   float64
windspeed                  float64
winddir                    float64
sealevelpressure           float64
cloudcover                 float64
visibility                 float64
solarradiation             float64
solarenergy                float64
uvindex                      int64
moonphase                  float64
conditions                  object
pm25                        object
dtype: object

In [10]:
merged25_df['pm25'] = merged25_df['pm25'].str.strip()
merged25_df['pm25'] = merged25_df['pm25'].replace(' ', np.nan)

merged25_df['pm25'] = pd.to_numeric(merged25_df['pm25'], errors='coerce')

merged25_df['pm25'] = merged25_df['pm25'].astype('Int64') 

In [11]:
# merged_df['pm25'].isna().sum()
merged25_df = merged25_df.dropna(subset=['pm25'])

In [14]:
merged25_df['conditions'].unique()
conditions = {'Clear': 1, 'Partially cloudy': 2, 'Rain, Partially cloudy': 3, 'Rain': 4, 
              'Overcast': 5, 'Rain, Overcast': 6}

merged25_df['conditions'] = merged25_df['conditions'].map(conditions)

In [ ]:
#merge 2025 data with existing data
merged_df = pd.read_csv('data/processed_data/merged_la_data.csv')
merged_df = pd.concat([merged_df, merged25_df], ignore_index=True)

In [16]:
merged25_df.dtypes

date                datetime64[ns]
tempmax                    float64
tempmin                    float64
temp                       float64
feelslikemax               float64
feelslikemin               float64
feelslike                  float64
dew                        float64
humidity                   float64
precip                     float64
precipprob                   int64
precipcover                float64
snow                         int64
snowdepth                    int64
windgust                   float64
windspeed                  float64
winddir                    float64
sealevelpressure           float64
cloudcover                 float64
visibility                 float64
solarradiation             float64
solarenergy                float64
uvindex                      int64
moonphase                  float64
conditions                 float64
pm25                         Int64
dtype: object

In [17]:
# merged_df.to_csv('data/processed_data/merged_la_data.csv', index=False)
merged25_df.to_csv('data/processed_data/merged25_la_data.csv', index=False)

# Add Historical Data to S3

In [13]:
bucket = sagemaker.Session().default_bucket()
prefix = "air-quality-proj"

s3_client = boto3.client('s3')

[07/01/25 11:36:37] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=872436;file://c:\Users\sadai\Desktop\AQI-Prediction\.venv\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=615571;file://c:\Users\sadai\Desktop\AQI-Prediction\.venv\lib\site-packages\botocore\credentials.py#1352\1352]8;;\

[07/01/25 11:36:39] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=641493;file://c:\Users\sadai\Desktop\AQI-Prediction\.venv\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=58974;file://c:\Users\sadai\Desktop\AQI-Prediction\.venv\lib\site-packages\botocore\credentials.py#1352\1352]8;;\

In [18]:
# # Upload each dataset to S3
local_path = 'data/processed_data/merged25_la_data.csv'
s3_path = f'{prefix}/data/merged25_la_data.csv'
    
s3_client.upload_file(
    local_path,
    bucket,
    s3_path
)

# Ingest Data into Sagemaker Feature Store

In [19]:
import boto3

# Create SageMaker client
sagemaker_client = boto3.client('sagemaker')

# List all feature groups
response = sagemaker_client.list_feature_groups()

# Print feature group names
for feature_group in response['FeatureGroupSummaries']:
    print(f"Feature Group Name: {feature_group['FeatureGroupName']}")
    print(f"Creation Time: {feature_group['CreationTime']}")
    print(f"Status: {feature_group['Status']}")
    print("---")


Feature Group Name: air-quality-features-08-14-56-40
Creation Time: 2025-04-08 10:56:50.530000-04:00


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:13                                                                                   │
│                                                                                                  │
│   10 for feature_group in response['FeatureGroupSummaries']:                                     │
│   11 │   print(f"Feature Group Name: {feature_group['FeatureGroupName']}")                       │
│   12 │   print(f"Creation Time: {feature_group['CreationTime']}")                                │
│ ❱ 13 │   print(f"Status: {feature_group['Status']}")                                             │
│   14 │   print("---")                                                                            │
│   15                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'Status'

In [20]:
sagemaker_session = sagemaker.Session()
role = 'arn:aws:iam::784376946367:role/sagemakerRole'

In [21]:
import s3fs
fs = s3fs.S3FileSystem(anon=False)
data = pd.read_csv('s3://sagemaker-us-east-1-784376946367/air-quality-proj/data/merged25_la_data.csv')

[07/01/25 12:30:48] INFO     Found credentials in shared credentials file: ~/.aws/credentials    ]8;id=240364;file://c:\Users\sadai\Desktop\AQI-Prediction\.venv\lib\site-packages\aiobotocore\credentials.py\credentials.py]8;;\:]8;id=906116;file://c:\Users\sadai\Desktop\AQI-Prediction\.venv\lib\site-packages\aiobotocore\credentials.py#670\670]8;;\

In [22]:
# Helper function to cast object types to string
def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == "object":
            data_frame[label] = data_frame[label].astype("str").astype("string")

In [23]:
# Cast object columns to string
cast_object_to_string(data)

In [24]:
# Handle timestamp - ensure it's float64 type
if 'timestamp' not in data.columns:
    data['timestamp'] = pd.Series([int(round(time.time()))] * len(data), dtype="float64")
# Add record identifier if not present
if 'record_id' not in data.columns:
    data['record_id'] = data.index.astype(str)

In [25]:
feature_group_name ='air-quality-features-08-14-56-40'#"air-quality-features-"+ strftime("%d-%H-%M-%S", gmtime())
feature_group = FeatureGroup(
    name=feature_group_name,
    sagemaker_session=sagemaker_session
)

In [26]:
# Load feature definitions directly from dataframe
feature_group.load_feature_definitions(data_frame=data)

[FeatureDefinition(feature_name='date', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None),
 FeatureDefinition(feature_name='tempmax', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='tempmin', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='temp', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='feelslikemax', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='feelslikemin', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='feelslike', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='dew', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(fea

In [ ]:
# Create the feature group
feature_group.create(
    s3_uri=f"s3://{sagemaker_session.default_bucket()}/air-quality-proj/data/feature_store",
    record_identifier_name="record_id",
    event_time_feature_name="timestamp",
    role_arn=role,
    enable_online_store=True
)

In [27]:
# Wait for feature group to become active
status = feature_group.describe().get("FeatureGroupStatus")
print(f"Initial status: {status}")

while status != "Active":
    print(f"Waiting for feature group to become active. Current status: {status}")
    time.sleep(5)
    status = feature_group.describe().get("FeatureGroupStatus")

print("Feature group is now active")

Initial status: Created
Waiting for feature group to become active. Current status: Created
Waiting for feature group to become active. Current status: Created


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:7                                                                                    │
│                                                                                                  │
│    4                                                                                             │
│    5 while status != "Active":                                                                   │
│    6 │   print(f"Waiting for feature group to become active. Current status: {status}")          │
│ ❱  7 │   time.sleep(5)                                                                           │
│    8 │   status = feature_group.describe().get("FeatureGroupStatus")                             │
│    9                                                                                             │
│   10 print("Feature group is now active")                                                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

In [28]:
#check for any nulls before ingesting data
data.isna().sum()

date                  0
tempmax               0
tempmin               0
temp                  0
feelslikemax          0
feelslikemin          0
feelslike             0
dew                   0
humidity              0
precip                0
precipprob            0
precipcover           0
snow                  0
snowdepth             0
windgust              0
windspeed             0
winddir               0
sealevelpressure      0
cloudcover            0
visibility            0
solarradiation        0
solarenergy           0
uvindex               0
moonphase             0
conditions          159
pm25                  0
timestamp             0
record_id             0
dtype: int64

In [29]:
# Ingest the data
feature_group.ingest(data_frame=data, wait=True)

[07/01/25 12:31:33] INFO     Started ingesting index %d to %d                                  ]8;id=200189;file://c:\Users\sadai\Desktop\AQI-Prediction\.venv\lib\site-packages\sagemaker\feature_store\feature_group.py\feature_group.py]8;;\:]8;id=745513;file://c:\Users\sadai\Desktop\AQI-Prediction\.venv\lib\site-packages\sagemaker\feature_store\feature_group.py#448\448]8;;\

IngestionManagerPandas(feature_group_name='air-quality-features-08-14-56-40', feature_definitions={'date': {'FeatureName': 'date', 'FeatureType': 'String'}, 'tempmax': {'FeatureName': 'tempmax', 'FeatureType': 'Fractional'}, 'tempmin': {'FeatureName': 'tempmin', 'FeatureType': 'Fractional'}, 'temp': {'FeatureName': 'temp', 'FeatureType': 'Fractional'}, 'feelslikemax': {'FeatureName': 'feelslikemax', 'FeatureType': 'Fractional'}, 'feelslikemin': {'FeatureName': 'feelslikemin', 'FeatureType': 'Fractional'}, 'feelslike': {'FeatureName': 'feelslike', 'FeatureType': 'Fractional'}, 'dew': {'FeatureName': 'dew', 'FeatureType': 'Fractional'}, 'humidity': {'FeatureName': 'humidity', 'FeatureType': 'Fractional'}, 'precip': {'FeatureName': 'precip', 'FeatureType': 'Fractional'}, 'precipprob': {'FeatureName': 'precipprob', 'FeatureType': 'Integral'}, 'precipcover': {'FeatureName': 'precipcover', 'FeatureType': 'Fractional'}, 'snow': {'FeatureName': 'snow', 'FeatureType': 'Integral'}, 'snowdepth': 

In [30]:
# Get detailed status
print(feature_group.describe())

# If there were ingestion errors, you can check them
print(feature_group.describe().get("FailureReason"))


{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:784376946367:feature-group/air-quality-features-08-14-56-40', 'FeatureGroupName': 'air-quality-features-08-14-56-40', 'RecordIdentifierFeatureName': 'record_id', 'EventTimeFeatureName': 'timestamp', 'FeatureDefinitions': [{'FeatureName': 'date', 'FeatureType': 'String'}, {'FeatureName': 'tempmax', 'FeatureType': 'Fractional'}, {'FeatureName': 'tempmin', 'FeatureType': 'Fractional'}, {'FeatureName': 'temp', 'FeatureType': 'Fractional'}, {'FeatureName': 'feelslikemax', 'FeatureType': 'Fractional'}, {'FeatureName': 'feelslikemin', 'FeatureType': 'Fractional'}, {'FeatureName': 'feelslike', 'FeatureType': 'Fractional'}, {'FeatureName': 'dew', 'FeatureType': 'Fractional'}, {'FeatureName': 'humidity', 'FeatureType': 'Fractional'}, {'FeatureName': 'precip', 'FeatureType': 'Fractional'}, {'FeatureName': 'precipprob', 'FeatureType': 'Integral'}, {'FeatureName': 'precipcover', 'FeatureType': 'Fractional'}, {'FeatureName': 'snow', 'FeatureType': 'In

In [6]:
# Initialize the Feature Store Runtime client
featurestore_runtime = boto3.client('sagemaker-featurestore-runtime')

# Specify the feature group name and record identifier
feature_group_name = 'air-quality-features-08-14-56-40'
record_identifier_value = str(1245)  # Convert to string

try:
    # Get the record
    response = featurestore_runtime.get_record(
        FeatureGroupName=feature_group_name,
        RecordIdentifierValueAsString=record_identifier_value,
    )
    
    # Print the record
    print("Retrieved record:")
    print(response['Record'])

except featurestore_runtime.exceptions.ResourceNotFound:  # Corrected exception name
    print(f"Record with ID {record_identifier_value} not found")
except Exception as e:
    print(f"Error retrieving record: {str(e)}")

Retrieved record:
[{'FeatureName': 'date', 'ValueAsString': '2023-06-13'}, {'FeatureName': 'tempmax', 'ValueAsString': '21.2'}, {'FeatureName': 'tempmin', 'ValueAsString': '16.7'}, {'FeatureName': 'temp', 'ValueAsString': '18.5'}, {'FeatureName': 'feelslikemax', 'ValueAsString': '21.2'}, {'FeatureName': 'feelslikemin', 'ValueAsString': '16.7'}, {'FeatureName': 'feelslike', 'ValueAsString': '18.5'}, {'FeatureName': 'dew', 'ValueAsString': '13.1'}, {'FeatureName': 'humidity', 'ValueAsString': '71.1'}, {'FeatureName': 'precip', 'ValueAsString': '0.0'}, {'FeatureName': 'precipprob', 'ValueAsString': '0'}, {'FeatureName': 'precipcover', 'ValueAsString': '0.0'}, {'FeatureName': 'snow', 'ValueAsString': '0'}, {'FeatureName': 'snowdepth', 'ValueAsString': '0'}, {'FeatureName': 'windgust', 'ValueAsString': '36.0'}, {'FeatureName': 'windspeed', 'ValueAsString': '12.8'}, {'FeatureName': 'winddir', 'ValueAsString': '252.9'}, {'FeatureName': 'sealevelpressure', 'ValueAsString': '1016.4'}, {'Feature

In [42]:
# List all feature groups
response = sagemaker_client.list_feature_groups()

# Print feature group names and creation times
print("Existing Feature Groups:")
for feature_group in response['FeatureGroupSummaries']:
    print(f"Name: {feature_group['FeatureGroupName']}")
    print(f"Creation Time: {feature_group['CreationTime']}")
    print(f"Status: {feature_group['FeatureGroupStatus']}")  # Changed from 'Status' to 'FeatureGroupStatus'
    print("---")


Existing Feature Groups:
Name: air-quality-features-08-14-56-40
Creation Time: 2025-04-08 10:56:50.530000-04:00
Status: Created
---


# Train/Val/Test Split

In [ ]:
train, test = train_test_split(merged_df, test_size=0.2, random_state=33, shuffle=False)
train, val = train_test_split(train, test_size=0.25, random_state=33, shuffle=False)

In [99]:
# pd.to_csv('data/processed_data/train.csv', index=False)
# pd.to_csv('data/processed_data/val.csv', index=False)
# pd.to_csv('data/processed_data/test.csv', index=False)

In [101]:
# Upload to S3
bucket = sagemaker.Session().default_bucket()
prefix = "air-quality-proj"

s3_client = boto3.client('s3')


In [104]:
# # Upload each dataset to S3
# for dataset in ['train', 'validation', 'test']:
#     local_path = f'data/processed_data/{dataset}.csv'
#     s3_path = f'{prefix}/data/{dataset}.csv'
    
#     s3_client.upload_file(
#         local_path,
#         bucket,
#         s3_path
#     )

In [6]:

# # Ensure all columns have the correct data types
# merged_data['city'] = merged_data['city'].astype(str)
# # merged_data['date'] = merged_data['date'].astype(str)  # Assuming date is stored as string
# merged_data['pm25'] = merged_data['pm25'].astype(float)
# merged_data['pm10'] = merged_data['pm10'].astype(float)
# merged_data['no2'] = merged_data['no2'].astype(float)
# merged_data['so2'] = merged_data['so2'].astype(float)
# merged_data['co'] = merged_data['co'].astype(float)
# merged_data['o3'] = merged_data['o3'].astype(float)